# Instruction-tuning LLM (or SFT, part of Multi-step fine-tuning)

This is notebook for adapting continually pretrained before LLM (DeepSeek-R1-Distill-Qwen-1.5B-medical-continual-pretrain-merged-f32) to medical domain by doing instruction tuning (using SFT form unsloth) on medical dataset of triples of question-CoT-answer. This notebook produces final Multi-step fine-tuned model for the project.

### Setup

In [ ]:
from IPython.display import clear_output

!pip install unsloth transformers datasets trl torch huggingface-hub wandb scikit-learn bitsandbytes accelerate
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
clear_output(wait=False)

In [2]:
import random
import numpy as np
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

SEED = 4242
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [3]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv('HF_TOKEN')

login(hf_token)

m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
import wandb

wandb_api = os.getenv('WANDB_API')
wandb.login(key=wandb_api)

run = wandb.init(
    project='Deepseek-R1-Qwen-1.5b msft on medical dataset, full 1 epoch v.0',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\milya\_netrc
wandb: Currently logged in as: miliusha2801 (miliusha2801-innopolis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

'cuda'

### Model loading and QLoRA setup

In [ ]:
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch


model_name = "MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-continual-pretrain-merged-f32"
max_seq_length = 4096
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-08 19:39:32 [__init__.py:256] Automatically detected platform cuda.


m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0. vLLM: 0.8.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
print(next(model.parameters()).dtype)

torch.bfloat16


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    use_rslora=True,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Dataset loading and preparation

In [9]:
from datasets import load_dataset

ds = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train[:20000]", trust_remote_code=True)

Generating train split: 100%|██████████| 25371/25371 [00:01<00:00, 16864.92 examples/s]


In [10]:
ds

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 20000
})

In [11]:
ds[0]

{'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. \n\nThe fact that she underwent a Q-tip test is intriguing too. This 

In [12]:
train_prompt_style = """
### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [13]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["Question"]
    thoughts = examples["Complex_CoT"]
    responses = examples["Response"]
    texts = []
    for question, thought, response in zip(questions, thoughts, responses):
        text = train_prompt_style.format(question, thought, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [14]:
ds_formatted = ds.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=["Question", "Complex_CoT", "Response"]
)

Map: 100%|██████████| 20000/20000 [00:00<00:00, 49718.81 examples/s]


In [15]:
ds_formatted[0]["text"]

"\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physic

In [16]:
from datasets import *

ds_splitted = ds_formatted.train_test_split(test_size=0.05, seed=SEED)

In [17]:
ds_splitted

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 19000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

### Instruction tuning (SFT)

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

finetune_name = "DeepSeek-R1-Distill-Qwen-1.5B-medical-msft-merged-f32"

training_args = UnslothTrainingArguments(
    output_dir=finetune_name,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    save_steps=200,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch_fused",
    lr_scheduler_type="cosine",
    warmup_steps=300,
    learning_rate=1e-4,
    num_train_epochs=1,
    weight_decay = 0.01,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    tf32=False,
    seed=SEED,
    report_to="wandb",
    hub_model_id=finetune_name,
    gradient_checkpointing=True,
)

In [26]:
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds_splitted["train"],
    eval_dataset=ds_splitted["test"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    args=training_args,
)

In [27]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,000 | Num Epochs = 1 | Total steps = 2,375
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768/5,000,000,000 (0.37% trained)


Step,Training Loss,Validation Loss
100,1.995800,1.769536
200,1.771500,1.740610
300,1.773100,1.732168
400,1.725000,1.716184
500,1.736100,1.705881
600,1.711000,1.697127
700,1.693600,1.695834
800,1.704100,1.680947
900,1.672500,1.686908
1000,1.717200,1.682304


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\peft\utils\other.py:1107: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-continual-pretrain-merged-f32/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002BDC420E190>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: c78732af-75c0-4ac0-829f-7b9bdef8dc4e)') - silently ignoring the lookup for the file config.json in MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-continual-pretrain-merged-f32.
  warnings.warn(
m:\python_projects\MedAlign-

### Saving model in HF Hub

In [3]:
import unsloth
from unsloth import FastLanguageModel
import torch

checkpoint_path = "./DeepSeek-R1-Distill-Qwen-1.5B-medical-msft-merged-f32/checkpoint-2375"
output_hub_model_name = "MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-msft-merged"
max_seq_length = 4096
dtype = None
load_in_4bit = False


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=checkpoint_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0. vLLM: 0.8.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
model = model.merge_and_unload()

model.push_to_hub(output_hub_model_name)
tokenizer.push_to_hub(output_hub_model_name)

Saved model to https://huggingface.co/MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-msft-merged


### Check fine-tuned LLM inference

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
from unsloth import FastLanguageModel

model_name = "MilyaShams/DeepSeek-R1-Distill-Qwen-1.5B-medical-msft-merged"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8192,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-09 15:53:25 [__init__.py:256] Automatically detected platform cuda.


m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0. vLLM: 0.8.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)
FastLanguageModel.for_inference(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((153

In [4]:
question = "What type of cement bonds to tooth structure, provides an anticariogenic effect, has a degree of translucency, and is non-irritating to the pulp?"

messages = [{"from": "human", "value": question}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=4096, use_cache=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

What type of cement bonds to tooth structure, provides an anticariogenic effect, has a degree of translucency, and is non-irritating to the pulp?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Which of the following is a type of cement that is used in the production of cement paste?
A. Matrix cement
B. Matrix cement
C. Matrix cement
D. Matrix cement
E. Matrix cement
F. Matrix cement
G. Matrix cement
H. Matrix cement
I. Matrix cement
J. Matrix cement

The following sentences are paired; for each pair, one sentence is the question and the other is the answer. The sentences are paired randomly so that you will not know which sentence is the question and which is the answer until you click the button.
The following sentences are paired; for each pair, one sentence is the question and the other is the a

KeyboardInterrupt: 